# 0. Machine Learning Preprocessing Notebook

The aim of this notebook is to use machine learning and deep learning to forecast future games between the two opposing teams. There are many external factors that influence the outcome of two opposing teams, such as player injuries, team momentum, substitutions, personal lives and more. However, the purpose of this notebook is an initial proof of concept. 

The first requirement is to remove all post-game features so that our model can train on. We can then split the table into home and away team, followed by additional feature engineering to aggregate each team to extract generalizable metrics for our ML model. We can then use probability estimates to predict the likelihood of each team winning.


In [27]:
# Dependencies
import pandas as pd
import numpy as np
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

In [5]:
# Read in modified dataframe
df_data = pd.read_csv('Resources/team_data_scored.csv')
df_data

,#,season_id,game_id,game_date,team_id_home,team_name_home,ftm_home,fta_home,ft_pct_home,oreb_home,...,pts_ot7_away,pts_ot8_away,pts_ot9_away,pts_ot10_away,offensive_rating_home,offensive_rating_away,defensive_rating_home,defensive_rating_away,efficiency_rating_home,efficiency_rating_away
0,1,21999,29900423,2000-01-02 00:00:00,1610612748,Miami Heat,27,32,0.844,12,...,NaN,NaN,NaN,NaN,5.717183,5.697563,4.288730,4.367902,5.463832,5.191898
1,3,21999,29900426,2000-01-03 00:00:00,1610612764,Washington Wizards,14,18,0.778,15,...,NaN,NaN,NaN,NaN,5.191389,4.648808,4.238598,3.928005,4.882802,4.648230
2,4,21999,29900429,2000-01-03 00:00:00,1610612762,Utah Jazz,16,20,0.800,12,...,NaN,NaN,NaN,NaN,7.225045,6.242841,4.232899,3.830245,5.573737,4.689307
3,5,21999,29900428,2000-01-03 00:00:00,1610612741,Chicago Bulls,9,17,0.529,16,...,NaN,NaN,NaN,NaN,7.110506,7.676224,4.326142,5.128922,3.844642,4.432885
4,6,21999,29900425,2000-01-03 00:00:00,1610612755,Philadelphia 76ers,28,41,0.683,11,...,NaN,NaN,NaN,NaN,5.047427,5.120270,3.807748,3.789897,5.685279,5.329089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25217,30484,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,0.0,0.0,0.0,0.0,NaN,5.182691,3.322732,3.399473,4.471639,3.423176
25218,30485,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,0.0,0.0,0.0,0.0,NaN,5.182691,3.322732,3.399473,4.471639,3.423176
25219,30486,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,0.0,0.0,0.0,0.0,NaN,5.182691,3.322732,3.399473,4.471639,3.423176
25220,30487,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,0.0,0.0,0.0,0.0,NaN,5.182691,3.322732,3.399473,4.471639,3.423176


# 1. Feature Engineering for ML


In [6]:
# OHE season_type
df_data.loc[df_data['season_type'] == "All Star", 'season_type'] = "All-Star"
season_set = pd.get_dummies(df_data['season_type'], prefix="season_type").astype('int')
df_data = pd.concat([df_data, season_set], axis=1)

In [7]:
df_data['home_wins'] = (df_data['pts_home'] > df_data['pts_away']).astype('int')
df_data['away_wins'] = (df_data['pts_home'] < df_data['pts_away']).astype('int')

In [8]:
df_data

,#,season_id,game_id,game_date,team_id_home,team_name_home,ftm_home,fta_home,ft_pct_home,oreb_home,...,defensive_rating_home,defensive_rating_away,efficiency_rating_home,efficiency_rating_away,season_type_All-Star,season_type_Playoffs,season_type_Pre Season,season_type_Regular Season,home_wins,away_wins
0,1,21999,29900423,2000-01-02 00:00:00,1610612748,Miami Heat,27,32,0.844,12,...,4.288730,4.367902,5.463832,5.191898,0,0,0,1,1,0
1,3,21999,29900426,2000-01-03 00:00:00,1610612764,Washington Wizards,14,18,0.778,15,...,4.238598,3.928005,4.882802,4.648230,0,0,0,1,1,0
2,4,21999,29900429,2000-01-03 00:00:00,1610612762,Utah Jazz,16,20,0.800,12,...,4.232899,3.830245,5.573737,4.689307,0,0,0,1,1,0
3,5,21999,29900428,2000-01-03 00:00:00,1610612741,Chicago Bulls,9,17,0.529,16,...,4.326142,5.128922,3.844642,4.432885,0,0,0,1,0,1
4,6,21999,29900425,2000-01-03 00:00:00,1610612755,Philadelphia 76ers,28,41,0.683,11,...,3.807748,3.789897,5.685279,5.329089,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25217,30484,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,3.322732,3.399473,4.471639,3.423176,1,0,0,0,0,1
25218,30485,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,3.322732,3.399473,4.471639,3.423176,1,0,0,0,0,1
25219,30486,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,3.322732,3.399473,4.471639,3.423176,1,0,0,0,0,1
25220,30487,32022,32200001,2023-02-19 00:00:00,1610616834,Team LeBron,0,0,NaN,13,...,3.322732,3.399473,4.471639,3.423176,1,0,0,0,0,1


In [9]:
# Split tables into home and away tables
standard_columns = ['#', 'season_id', 'game_id', 'game_date', 'times_tied', 'lead_changes', 'season_type_All-Star', 'season_type_Playoffs', 'season_type_Pre Season', 'season_type_Regular Season',]
home_columns = standard_columns + [col for col in df_data.columns if "home" in col]
away_columns = standard_columns + [col for col in df_data.columns if "away" in col]

df_home = df_data[home_columns]
df_away = df_data[away_columns]

In [10]:
df_away

,#,season_id,game_id,game_date,times_tied,lead_changes,season_type_All-Star,season_type_Playoffs,season_type_Pre Season,season_type_Regular Season,...,pts_ot5_away,pts_ot6_away,pts_ot7_away,pts_ot8_away,pts_ot9_away,pts_ot10_away,offensive_rating_away,defensive_rating_away,efficiency_rating_away,away_wins
0,1,21999,29900423,2000-01-02 00:00:00,3.0,7.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,5.697563,4.367902,5.191898,0
1,3,21999,29900426,2000-01-03 00:00:00,6.0,11.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,4.648808,3.928005,4.648230,0
2,4,21999,29900429,2000-01-03 00:00:00,2.0,0.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,6.242841,3.830245,4.689307,0
3,5,21999,29900428,2000-01-03 00:00:00,0.0,0.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,7.676224,5.128922,4.432885,1
4,6,21999,29900425,2000-01-03 00:00:00,12.0,11.0,0,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,5.120270,3.789897,5.329089,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25217,30484,32022,32200001,2023-02-19 00:00:00,14.0,24.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.182691,3.399473,3.423176,1
25218,30485,32022,32200001,2023-02-19 00:00:00,14.0,24.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.182691,3.399473,3.423176,1
25219,30486,32022,32200001,2023-02-19 00:00:00,14.0,24.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.182691,3.399473,3.423176,1
25220,30487,32022,32200001,2023-02-19 00:00:00,14.0,24.0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.182691,3.399473,3.423176,1


In [11]:
# Drop pts_ot greater than 5
cols_to_drop = df_away.filter(regex=r'pts_ot[5-9][0]?_.*').columns
df_away.drop(cols_to_drop, inplace=True, axis=1)
cols_to_drop = df_home.filter(regex=r'pts_ot[5-9][0]?_.*').columns
df_home.drop(cols_to_drop, inplace=True, axis=1)

/var/folders/_j/g4pw0jf17gq2w8c26hbhq5l80000gn/T/ipykernel_13216/1807922910.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away.drop(cols_to_drop, inplace=True, axis=1)
/var/folders/_j/g4pw0jf17gq2w8c26hbhq5l80000gn/T/ipykernel_13216/1807922910.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home.drop(cols_to_drop, inplace=True, axis=1)


In [21]:
# Modify tables
tables = [df_home, df_away]
labels = ['home', 'away']
teams_dfs = dict()
for table, label in zip(tables, labels):
    teams_dfs[f'{label}'] = table.groupby([f'team_id_{label}', f'team_name_{label}']).agg({'game_id': 'count',
                                                                   f'{label}_wins': 'sum',
                                                                   'season_type_Playoffs': 'sum', 
                                                                   'season_type_Pre Season': 'sum', 
                                                                   'season_type_Regular Season': 'sum',
                                                                   f'offensive_rating_{label}': 'std',
                                                                   f'defensive_rating_{label}': 'std',
                                                                   f'efficiency_rating_{label}': 'std',
                                                                   }).reset_index()

In [22]:
df_teams = pd.merge(left=teams_dfs['home'], right=teams_dfs['away'], left_on='team_name_home', right_on='team_name_away', how='inner')
df_teams.drop(['team_id_away', 'team_name_away', 'game_id_y', 'season_type_Playoffs_y', 'season_type_Pre Season_y', 'season_type_Regular Season_y'], axis=1, inplace=True)

In [23]:
df_teams

,team_id_home,team_name_home,game_id_x,home_wins,season_type_Playoffs_x,season_type_Pre Season_x,season_type_Regular Season_x,offensive_rating_home,defensive_rating_home,efficiency_rating_home,away_wins,offensive_rating_away,defensive_rating_away,efficiency_rating_away
0,12315,Madrid Real Madrid,2,1,0,2,0,0.640057,0.319421,0.589885,0,1.013158,0.008522,0.060430
1,12325,Flamengo Flamengo,1,0,0,1,0,NaN,NaN,NaN,0,0.134936,0.052876,0.019508
2,1610612737,Atlanta Hawks,833,465,52,24,757,0.941819,0.285418,0.390352,281,0.941177,0.291098,0.393337
3,1610612738,Boston Celtics,887,564,104,28,755,1.020028,0.305799,0.393096,421,0.974328,0.290738,0.370448
4,1610612739,Cleveland Cavaliers,865,516,56,37,772,0.967208,0.316984,0.384174,328,0.890643,0.283691,0.403230
5,1610612740,New Orleans Hornets,289,171,15,6,268,0.856985,0.293772,0.372723,103,0.916470,0.296634,0.398642
6,1610612740,New Orleans Pelicans,363,197,9,15,339,1.018033,0.281485,0.360425,139,0.987429,0.261812,0.367699
7,1610612740,New Orleans/Oklahoma City Hornets,63,34,0,0,63,0.863620,0.302212,0.397076,24,0.828142,0.261219,0.365062
8,1610612741,Chicago Bulls,850,465,39,37,774,0.944288,0.325532,0.383354,308,0.993369,0.290171,0.390472
9,1610612742,Dallas Mavericks,875,590,62,26,787,0.983479,0.311442,0.400474,444,0.981477,0.303732,0.402548


In [24]:
# Rename Columns
df_teams.columns = ['team_id', 'team_name', 'games_since_2000', 'home_wins', 'playoff_plays', 'preseason_plays', 'regseason_plays', 'off_rating_home', 'def_rating_home', 'eff_rating_home', 'away_wins', 'off_rating_away', 'def_rating_away', 'eff_rating_away']

df_teams

,team_id,team_name,games_since_2000,home_wins,playoff_plays,preseason_plays,regseason_plays,off_rating_home,def_rating_home,eff_rating_home,away_wins,off_rating_away,def_rating_away,eff_rating_away
0,12315,Madrid Real Madrid,2,1,0,2,0,0.640057,0.319421,0.589885,0,1.013158,0.008522,0.060430
1,12325,Flamengo Flamengo,1,0,0,1,0,NaN,NaN,NaN,0,0.134936,0.052876,0.019508
2,1610612737,Atlanta Hawks,833,465,52,24,757,0.941819,0.285418,0.390352,281,0.941177,0.291098,0.393337
3,1610612738,Boston Celtics,887,564,104,28,755,1.020028,0.305799,0.393096,421,0.974328,0.290738,0.370448
4,1610612739,Cleveland Cavaliers,865,516,56,37,772,0.967208,0.316984,0.384174,328,0.890643,0.283691,0.403230
5,1610612740,New Orleans Hornets,289,171,15,6,268,0.856985,0.293772,0.372723,103,0.916470,0.296634,0.398642
6,1610612740,New Orleans Pelicans,363,197,9,15,339,1.018033,0.281485,0.360425,139,0.987429,0.261812,0.367699
7,1610612740,New Orleans/Oklahoma City Hornets,63,34,0,0,63,0.863620,0.302212,0.397076,24,0.828142,0.261219,0.365062
8,1610612741,Chicago Bulls,850,465,39,37,774,0.944288,0.325532,0.383354,308,0.993369,0.290171,0.390472
9,1610612742,Dallas Mavericks,875,590,62,26,787,0.983479,0.311442,0.400474,444,0.981477,0.303732,0.402548


In [25]:
# Drop rows with less than 50 games
df_teams = df_teams[df_teams['games_since_2000'] >= 100]

df_teams

,team_id,team_name,games_since_2000,home_wins,playoff_plays,preseason_plays,regseason_plays,off_rating_home,def_rating_home,eff_rating_home,away_wins,off_rating_away,def_rating_away,eff_rating_away
2,1610612737,Atlanta Hawks,833,465,52,24,757,0.941819,0.285418,0.390352,281,0.941177,0.291098,0.393337
3,1610612738,Boston Celtics,887,564,104,28,755,1.020028,0.305799,0.393096,421,0.974328,0.290738,0.370448
4,1610612739,Cleveland Cavaliers,865,516,56,37,772,0.967208,0.316984,0.384174,328,0.890643,0.283691,0.403230
5,1610612740,New Orleans Hornets,289,171,15,6,268,0.856985,0.293772,0.372723,103,0.916470,0.296634,0.398642
6,1610612740,New Orleans Pelicans,363,197,9,15,339,1.018033,0.281485,0.360425,139,0.987429,0.261812,0.367699
8,1610612741,Chicago Bulls,850,465,39,37,774,0.944288,0.325532,0.383354,308,0.993369,0.290171,0.390472
9,1610612742,Dallas Mavericks,875,590,62,26,787,0.983479,0.311442,0.400474,444,0.981477,0.303732,0.402548
10,1610612743,Denver Nuggets,835,540,54,17,764,1.017952,0.298189,0.377776,350,0.936083,0.273265,0.393613
11,1610612744,Golden State Warriors,861,566,74,28,759,0.956987,0.293614,0.359051,351,1.025168,0.287435,0.371499
12,1610612745,Houston Rockets,840,527,56,28,756,1.005329,0.302915,0.370538,390,0.935364,0.286913,0.390722


1. Calculating proportion of wins, season plays

In [26]:
# Calculating additional metrics
df_teams['home_win_rate'] = ((df_teams['home_wins'])/ df_teams['games_since_2000'])
df_teams['away_win_rate'] = ((df_teams['away_wins'])/ df_teams['games_since_2000'])
df_teams['season_playoff_pct'] = (df_teams['playoff_plays'] /df_teams['games_since_2000']) /2
df_teams['season_preseason_pct'] = (df_teams['preseason_plays'] /df_teams['games_since_2000']) /2
df_teams['season_regular_pct'] = (df_teams['regseason_plays'] /df_teams['games_since_2000']) /2

df_teams.drop(['games_since_2000', 'home_wins', 'away_wins', 'playoff_plays', 'preseason_plays', 'regseason_plays'], axis=1, inplace=True)

/var/folders/_j/g4pw0jf17gq2w8c26hbhq5l80000gn/T/ipykernel_13216/2972029654.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams['home_win_rate'] = ((df_teams['home_wins'])/ df_teams['games_since_2000'])
/var/folders/_j/g4pw0jf17gq2w8c26hbhq5l80000gn/T/ipykernel_13216/2972029654.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_teams['away_win_rate'] = ((df_teams['away_wins'])/ df_teams['games_since_2000'])
/var/folders/_j/g4pw0jf17gq2w8c26hbhq5l80000gn/T/ipykernel_13216/2972029654.py:4: Settin

In [19]:
df_teams = df_teams.reset_index()

In [76]:
# Simulate game
def get_team_data(home_team, away_team):
    home_row = df_teams[df_teams['team_name'] == home_team.title()].reset_index()
    away_row = df_teams[df_teams['team_name'] == away_team.title()].reset_index()
    home_offensive_rating = home_row.at[0, 'off_rating_home']
    away_defensive_rating = away_row.at[0, 'def_rating_away']
    home_defensive_rating = home_row.at[0, 'off_rating_home']
    away_offensive_rating = away_row.at[0, 'def_rating_away']
    home_eff_rating =home_row.at[0, 'eff_rating_home']
    away_eff_rating = away_row.at[0, 'eff_rating_away']
    print(away_eff_rating)
    home_score = (home_offensive_rating - away_defensive_rating) 
    away_score = (home_defensive_rating - away_offensive_rating)

    return home_eff_rating, away_eff_rating, home_score, away_score
    
def simulate_matchup(home_team, away_team, simulations=100000):
    home_eff_rating, away_eff_rating, home_score, away_score = get_team_data(home_team, away_team)
    home_scores = np.random.normal(home_eff_rating, home_score, simulations)
    away_scores = np.random.normal(away_eff_rating, away_score, simulations)
    home_win_prob = np.sum(home_scores > away_scores)/ simulations
    return home_win_prob

In [77]:
results = simulate_matchup("Houston Rockets", "Boston Celtics")

0.37044814361034734
